In [ ]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate

In [ ]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from trainer import PointNetTrainer


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

In [ ]:
# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss


train_data = ModelNet(classes, DatasetType.TRAIN)
validation_data = ModelNet(classes, DatasetType.VALIDATION)
test_data = ModelNet(classes, DatasetType.TEST)

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

if DEVICE == "cuda":
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
else:
    scheduler = None
    
# TODO: Más adelante usar alpha para clases imbalanceadas

In [ ]:
classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

base_trainer = PointNetTrainer(
    name="pointnet_base",
    model=classifier,
    optimizer=optim.Adam(classifier.parameters(), lr=learning_rate),
    scheduler=scheduler,
    criterion=PointNetLoss(alpha=None, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
    device=DEVICE,
    train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
    val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
    checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
    checkpoint_freq=25
)

In [ ]:
loss_dict, best_epoch, best_loss, best_acc = base_trainer.fit(epochs=500)
save_loss_dict(loss_dict)
print(f"Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")

# Dataset Testeo

In [ ]:
with torch.no_grad():
    classifier = classifier.eval()
    correct = 0
    
    for pcds, labels in test_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)
        
        # Hacer predicciones
        out, _, _ = classifier(pcds)
    
        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas, acumuladas
        correct += pred_choice.eq(labels.data).cpu().sum().item()

    test_acc = correct / float(len(test_data))
    print("Test Acc:\t", test_acc)

In [ ]:
def test_it(path):
    classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
    classifier.load_state_dict(torch.load(path))

    with torch.no_grad():
        classifier = classifier.eval()
        correct = 0
        
        for pcds, labels in test_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, _ = classifier(pcds)
        
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas, acumuladas
            correct += pred_choice.eq(labels.data).cpu().sum().item()

        test_acc = correct / float(len(test_data))
        print(f"{path}\t\t", test_acc)

# Tests
for i in range(25, 500 + 1, 25):
    path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(i).zfill(4)}.pth")
    test_it(path)
path = os.path.join(CHECKPOINT_DIR, "best_model.pth")
test_it(path)